In [8]:
import numpy as np

In [9]:
kps = np.ones((300, 17, 3)).astype(np.float32)
kps[:, 13:, 2] = 0.1

In [10]:
kps[0]

array([[1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 0.1],
       [1. , 1. , 0.1],
       [1. , 1. , 0.1],
       [1. , 1. , 0.1]], dtype=float32)

In [11]:
def create_mask(kps):
    mask = np.where(
        kps[:, :, 2] < 0.2, -1e10, 0.0
    )  # -inf to nan in softmax of attention module
    seq_len, points = mask.shape
    mask = np.repeat(mask, 2, axis=1).reshape(seq_len, points, 2)
    return mask

In [12]:
mask = create_mask(kps)

In [14]:
mask[-1]

array([[ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [-1.e+10, -1.e+10],
       [-1.e+10, -1.e+10],
       [-1.e+10, -1.e+10],
       [-1.e+10, -1.e+10]])